In [2]:
#pip install youtube-search-python

In [2]:
#from youtubesearchpython.__future__ import VideosSearch
from youtubesearchpython import VideosSearch
import pandas as pd
from os import listdir
from os import remove
import csv
import time
import asyncio

pd.set_option('display.max_rows', None)

In [3]:
def getMetadata(video, file_name):
    videosSearch = VideosSearch(video, limit = 1)
    videosResult = videosSearch.result()
    if videosResult and "result" in videosResult and len(videosResult["result"]) > 0:
        title = videosResult['result'][0]['title']
        duration = videosResult['result'][0]['duration']

        view_count_txt = videosResult['result'][0]['viewCount']['text']
        view_count = 0
        if view_count_txt and (view_count_txt.isnumeric()) == True:
            view_count = int(view_count_txt.replace(",", "").split()[0])

        width = 0
        height = 0

        description_snippet_txt = videosResult['result'][0]['descriptionSnippet']
        description_snippet = ""
        if description_snippet_txt:
            description_snippet = description_snippet_txt[0]['text']

        channel_id = videosResult['result'][0]['channel']['id']
        channel_title = videosResult['result'][0]['channel']['name']
        
        published_time_txt = videosResult['result'][0]['publishedTime']
        published_time = ""
        if published_time_txt:
            published_time = published_time_txt
        
        status = "Success"

        with open(file_name, mode='a') as out_file:
            result_writer = csv.writer(out_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
            result_writer.writerow([video, title, duration, view_count, width, height, description_snippet, channel_id, channel_title, published_time, status])

In [8]:
# Create output file with header
date = time.strftime('%Y-%m-%d_%H_%M_%S')
file_name = "processed/training_videos_metadata" + date + ".csv"

# Get videos
filepaths = ["processed/" + f for f in listdir("processed/") if f.startswith("training_videos2022-01-10_15:18:33")]
df_videos = pd.concat(map(pd.read_csv, filepaths))

# Get already processed videos
filepaths = ["processed/" + f for f in listdir("processed/") if f.startswith("training_videos_metadata2022-01-10_15_33_33.csv")]
df_processed = pd.concat(map(pd.read_csv, filepaths))

# Create dataframe with unique videos
df_new_unique_videos = pd.merge(df_videos, df_processed, on="video", indicator=True, how='outer').query('_merge=="left_only"').drop('_merge', axis=1)

with open(file_name, mode='a') as out_file:
    result_writer = csv.writer(out_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    result_writer.writerow(["video", "title", "duration", "view_count", "width", "height", "description_snippet", "channel_id", "channel_title", "published_time", "status"])

print("New videos:", len(df_new_unique_videos))

i = 0
for index, row in df_new_unique_videos.iterrows():
    getMetadata(row["video"], file_name)
    if i % 500 == 0: 
        print(i)
    i += 1

New videos: 83
0
